In [49]:
from sqlalchemy import create_engine
from application import *
from application.models import Metadata
from sqlalchemy.sql.expression import func

In [2]:
meta = Metadata().query.filter(Metadata.nyt_id=='4fc08c4145c1498b0d35c415').one_or_none()
meta

Review u'4fc08c4145c1498b0d35c415'

In [50]:
rows = Metadata().query.filter(Metadata.review_type == "needs_audit").filter(Metadata.headline.like("%$%")).all()
len(rows)

3349

In [52]:
# load nltk names
from nltk.corpus import names
from collections import Counter

In [53]:
m = [i.lower() for i in names.words('male.txt')]
f = [i.lower() for i in names.words('female.txt')]
both = f+m
overlap_names = [i[0] for i in Counter(both).most_common() if i[1] == 2] 

#remove repeats from f and m
m = [i for i in m if i not in overlap_names]
f = [i for i in f if i not in overlap_names]

md = {key: 'male' for key in m}
fd = {key: 'female' for key in f}

#get names in both lists

ovd = {key: 'both' for key in overlap_names}
ovd['sandy']

'both'

In [54]:
matches = []
# match male and female names in rows
for row in rows:
    headline_tokens = row.headline.split(" ")
    match_names = {"male":[], "female":[], "either":[]}
    for h in headline_tokens:
        # find 'female', 'male', 'both', 'either'
        try:
            md[h.lower()]
            match_names["male"].append(h.lower())
        except:
            pass
        try:
            fd[h.lower()]
            match_names["female"].append(h.lower())
        except:
            pass
        # handle names in both lists
        try:
            ovd[h.lower()]
            match_names["either"].append(h.lower())
        except:
            pass
    # return tuples of names, category, nyt_id
    matches.append((row.nyt_id, match_names))

In [55]:
'''
N 0 0 = male
0 N 0 = multi
0 0 N = female
N N 0 = male or multi
0 N N = female or multi
N 0 N = multi
0 0 0 = no names recognized
N N N = multi
'''
matches[0][1].keys()

['male', 'either', 'female']

In [56]:
match_counts = [[len(i) for i in h[1].values()] for h in matches]
probably_female = []
for y, m in enumerate(match_counts):
    if m[0] == 0:
        probably_female.append(matches[y][0])
probably_male = []
for y, m in enumerate(match_counts):
    if m[2] == 0:
        probably_male.append(matches[y][0])

In [57]:
len(probably_female)

900

In [58]:
len(probably_male)

2608

In [59]:
900.0/2608

0.3450920245398773

In [73]:
import pandas as pd
df = pd.DataFrame(probably_female, columns=['nyt_id'])
df.to_csv('probably_female_reviews.csv')

In [63]:
prob_fem_rows = Metadata.query.filter(Metadata.nyt_id.in_(probably_female)).all()

In [65]:
for p in prob_fem_rows:
    p.review_type = 'needs_audit_probably_female'

In [67]:
db.session.commit()

In [68]:
prob_m_rows = Metadata.query.filter(Metadata.nyt_id.in_(probably_male)).all()
for p in prob_m_rows:
    p.review_type = 'needs_audit_probably_male'

In [70]:
db.session.commit()